In [98]:
import export_objects as eo
import analyze_subroutines as alsub
import fortran_modules as fm
import utilityFunctions as uf
import importlib
import os
import csv

import database.connect as cnt
import database.utilities as util

importlib.reload(eo)
importlib.reload(alsub)
importlib.reload(fm)
importlib.reload(uf)
importlib.reload(cnt)
importlib.reload(util)

mod_dict, sub_dict, type_dict = {}, {}, {}
mod_dict, sub_dict, type_dict = eo.unpickle_unit_test(mod_dict, sub_dict, type_dict)


In [169]:
# connect to database
cur, conn = cnt.connect()

In [150]:
filename = "out.txt"
# use file to create csv for module table
csv_folder = "CSV"
os.makedirs(csv_folder, exist_ok=True)

In [100]:
# write all module into to file
def write_modules_info_to_text(filename=filename):
    with open(filename, "w") as file:
        for keys in mod_dict.keys():
            mod_dict[keys].export_module_info(ofile=file)

In [101]:
write_modules_info_to_text(filename)

In [27]:


def TABLE_modules(filename, csv_file="TABLE_module"):
    
    """
    TABLE SETUP:
    +-------------+--------------+------+-----+---------+----------------+
    | Field       | Type         | Null | Key | Default | Extra          |
    +-------------+--------------+------+-----+---------+----------------+
    | module_id   | int          | NO   | PRI | NULL    | auto_increment |
    | module_name | varchar(100) | NO   | UNI | 0       |                |
    +-------------+--------------+------+-----+---------+----------------+
    
    INITIALIZE NULL WITH COMMAND
    
    INSERT INTO modules (module_name) VALUES ('NULL');
    UPDATE modules SET module_id=0 where module_id=1;
    
    """
    
    # MYSQL will automatically add module_id column
    
    cols = ["module_name"]
    
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        
        with open(filename, "r") as reader:
            for line in reader:
                if "Module:" in line:
                    # Module: <module>
                    module = line.split(" ")[-1].strip()
                    
                    writer.writerow([module])
    

In [30]:
csv_file = "CSV/TABLE_module.csv"
# TABLE_modules(filename, csv_file=csv_file)
util.insert(cur, conn, "modules", csv_file)

In [118]:
def TABLE_module_dependency(filename, csv_file="TABLE_module_dependency"):
    
    """
    TABLE SETUP
    +---------------+--------------+------+-----+---------+----------------+
    | Field         | Type         | Null | Key | Default | Extra          |
    +---------------+--------------+------+-----+---------+----------------+
    | dependency_id | int          | NO   | PRI | NULL    | auto_increment |
    | module_id     | int          | YES  | MUL | NULL    |                |
    | dep_module_id | int          | YES  | MUL | NULL    |                |
    | object_used   | varchar(100) | YES  |     | NULL    |                |
    +---------------+--------------+------+-----+---------+----------------+
    """
    
    cols = ["module_id", "dep_module_id","object_used"]
    s = set()
    with open(f"{csv_folder}/{csv_file}.csv","w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        with open(filename, "r") as reader:
            cur_module = ""
            cur_dep = ""
            for line in reader:
                if "Module:" in line:
                    # Module: <module>
                    cur_module = line.split(" ")[-1].strip()
                elif "None" in line:
                    writer.writerow([cur_module, "NULL", "NULL"])
                elif "->" in line:
                    cur_dep = line.split(" ")[-1].strip()
                elif line[:4] == "   -":
                    line = line.strip().split()[1:]
                    if "=>" in line:
                        line = line[2:]
                    if len(line) == 1:
                        writer.writerow([cur_module, cur_dep, line[0].strip()])

                    if len(line) == 3:
                        writer.writerow([cur_module, cur_dep, line[1].strip()])
                    s.add(len(line))

                # if "Variables:" in line:
                #     # [type, member_type, dim]
                #     line = line.split(" ")
                #     writer.writerow(["","",line[1]])
                    
                    
                    
                    
                   

In [223]:
cur.execute(
    "insert into test (txt) values ('googoo')"
)
conn.commit()

In [119]:

TABLE_module_dependency(filename)

In [45]:
csv_file = "CSV/TABLE_module_dependency.csv"

"""
insert into module_dependency (module_id, dep_module_id, object_used) 
select m1.module_id, m2.module_id, 'test' 
from modules m1 
join modules m2 on m2.module_name = 'vegetationtype' 
where m1.module_name = 'vegetationpropertiestype';
"""

"\ninsert into module_dependency (module_id, dep_module_id, object_used) \nselect m1.module_id, m2.module_id, 'test' \nfrom modules m1 \njoin modules m2 on m2.module_name = 'vegetationtype' \nwhere m1.module_name = 'vegetationpropertiestype';\n"

In [120]:
util.insert(cur, 
            conn, 
            "module_dependency", 
            csv_file=csv_file, 
            join=True, 
            ref_table="modules", 
            ref_cols=["module_name","module_name",""], 
            dep_col_names=["module_id","module_id",""],
            dep_cols=[1,1,0])


In [151]:
filename= "types_info.txt"
def write_types_info_to_text(filename=filename):
    with open(filename, "w") as file:
        for i in type_dict.keys():
            type_dict[i].export_derived_type(ofile=file)

In [149]:
write_types_info_to_text(filename)

In [161]:
def TABLE_user_types(filename, csv_file='TABLE_user_types'):
    """
    TABLE SETUP
    +----------------+--------------+------+-----+---------+----------------+
    | Field          | Type         | Null | Key | Default | Extra          |
    +----------------+--------------+------+-----+---------+----------------+
    | user_type_id   | int          | NO   | PRI | NULL    | auto_increment |
    | module_id      | int          | NO   | MUL | 0       |                |
    | user_type_name | varchar(100) | NO   | UNI | 0       |                |
    +----------------+--------------+------+-----+---------+----------------+
    """
    cols = ["module_id", "user_type_name"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        with open(filename, 'r') as f:
            type = ""
            mod = ""
            for line in f:
                if "Type:" in line:
                    type = line.split()[-1].strip()
                elif "Mod:" in line:
                    mod = line.split()[-1].strip()
                    writer.writerow([mod, type])


In [162]:
TABLE_user_types(filename)

In [ ]:
util.insert(cur, 
            conn, 
            "user_types", 
            "CSV/TABLE_user_types.csv", 
            join=True, 
            ref_table="modules",
            ref_cols=["module_name",""],
            dep_col_names=["module_id",""],
            dep_cols=[1,0])

In [176]:
def TABLE_user_type_instances(filename, csv_file='TABLE_user_type_instances'):
    
    """
    TABLE SETUP
    +------------------+--------------+------+-----+---------+----------------+
    | Field            | Type         | Null | Key | Default | Extra          |
    +------------------+--------------+------+-----+---------+----------------+
    | instance_id      | int          | NO   | PRI | NULL    | auto_increment |
    | instance_type_id | int          | NO   | MUL | 0       |                |
    | instance_name    | varchar(100) | NO   |     |         |                |
    +------------------+--------------+------+-----+---------+----------------+
    """
    
    cols = ["instance_type_id", "instance_name"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        with open(filename, 'r') as f:
            type = ""
            
            for line in f:
                if "Type:" in line:
                    type = line.split()[-1].strip()
                elif "i:" in line:
                    instance = line.split()[2].strip()
                    writer.writerow([type, instance])

In [177]:
filename= "types_info.txt"
TABLE_user_type_instances(filename)

In [ ]:
util.insert(cur,
            conn,
            "user_type_instances",
            "CSV/TABLE_user_type_instances.csv",
            join=True,
            ref_table="user_types",
            ref_cols=["user_type_name",""],
            dep_col_names=["user_type_id",""],
            dep_cols=[1,0])

In [199]:
def TABLE_type_defines(filename, csv_file="TABLE_type_defines"):
    import csv
        with open(csv_file, 'w') as file:
            writer = csv.writer(file, delimiter=",")
            writer.writerow(['subroutine_parent', "module_parent", "subroutine_child", "module_child", "arg"])

In [227]:
import export_objects as eo
import analyze_subroutines as alsub
import fortran_modules as fm
import utilityFunctions as uf
import DerivedType as dt
import importlib
import os
import csv

import database.connect as cnt
import database.utilities as util

importlib.reload(eo)
importlib.reload(alsub)
importlib.reload(fm)
importlib.reload(uf)
importlib.reload(dt)
importlib.reload(cnt)
importlib.reload(util)

mod_dict, sub_dict, type_dict = {}, {}, {}
mod_dict, sub_dict, type_dict = eo.unpickle_unit_test(mod_dict, sub_dict, type_dict)


In [228]:
for i in sub_dict.keys():
    sub_dict[i].export_subroutine_info()

Sub: laketemperature
 Mod: laketemperaturemod
Sub: soilthermprop_lake
 Mod: laketemperaturemod
Sub: phasechange_lake
 Mod: laketemperaturemod
Sub: get_clump_bounds_new
 Mod: decompmod
Sub: get_clump_bounds_old
 Mod: decompmod
Sub: get_proc_bounds_new
 Mod: decompmod
Sub: get_proc_bounds_old
 Mod: decompmod
Sub: get_proc_total
 Mod: decompmod
Sub: get_proc_global
 Mod: decompmod
Sub: get_proc_total_ghosts
 Mod: decompmod
Sub: init
 Mod: ch4mod
Sub: initallocate
 Mod: ch4mod
Sub: inithistory
 Mod: ch4mod
Sub: initcold
 Mod: ch4mod
Sub: restart
 Mod: ch4mod
Sub: readch4params
 Mod: ch4mod
Sub: ch4
 Mod: ch4mod
Sub: ch4_prod
 Mod: ch4mod
Sub: ch4_oxid
 Mod: ch4mod
Sub: ch4_aere
 Mod: ch4mod
Sub: siteoxaere
 Mod: ch4mod
Sub: ch4_ebul
 Mod: ch4mod
Sub: ch4_tran
 Mod: ch4mod
Sub: get_jwt
 Mod: ch4mod
Sub: ch4_annualupdate
 Mod: ch4mod
Sub: initcoldghost
 Mod: soilstatetype
Sub: col_pp_init
 Mod: columntype
Sub: col_pp_clean
 Mod: columntype
Sub: col_es_init
 Mod: columndatatype
Sub: col_es_re

In [219]:
import mod_config

x = (mod_config.home_dir + sub_dict["laketemperature"].filepath.split("/",6)[-1])

fm.get_module_name_from_file(x)

# fm.get_module_name_from_file("/mnt/c/Users/mungs/Desktop/SPEL_OpenACC/scripts/../../repo/E3SM/components/elm/src/biogeophys/LakeTemperatureMod.F90")
# fm.get_module_name_from_file("/home/mrgex/SPEL_Openacc/scripts/../../repo/E3SM/components/elm/src/biogeophys/LakeTemperatureMod.F90")

(1, 'laketemperaturemod')